In [ ]:
import torch
print('PyTorch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU name:', torch.cuda.get_device_name(0))
else:
    print('No GPU detected. Go to Runtime -> Change runtime type -> GPU')

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU name: Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATASET_ROOT = '/content/drive/MyDrive/Sign_detection'
TRAIN_IMAGES = DATASET_ROOT + '/train/'
VALID_IMAGES = DATASET_ROOT + '/valid/'
TRAIN_JSON = DATASET_ROOT + '/train/_annotations.coco.json'
VALID_JSON = DATASET_ROOT + '/valid/_annotations.coco.json'

In [ ]:
print('Train images:', TRAIN_IMAGES)
print('Valid images:', VALID_IMAGES)
print('Train JSON:', TRAIN_JSON)
print('Valid JSON:', VALID_JSON)

Train images: /content/drive/MyDrive/Sign_detection/train/
Valid images: /content/drive/MyDrive/Sign_detection/valid/
Train JSON: /content/drive/MyDrive/Sign_detection/train/_annotations.coco.json
Valid JSON: /content/drive/MyDrive/Sign_detection/valid/_annotations.coco.json


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17582, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17582 (delta 0), reused 0 (delta 0), pack-reused 17579 (from 2)
Receiving objects: 100% (17582/17582), 16.83 MiB | 14.42 MiB/s, done.
Resolving deltas: 100% (11978/11978), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.2 MB/s eta 0:00:00


In [ ]:
import json
from pathlib import Path
from collections import defaultdict
from PIL import Image


In [ ]:
def coco_to_yolo_and_save(images_dir, json_path, dataset_root=None, save_empty=False):

    images_dir = Path(images_dir)
    json_path = Path(json_path)
    if dataset_root is None:
        dataset_root = images_dir.parent
    else:
        dataset_root = Path(dataset_root)

    assert images_dir.exists(), f"Images dir not found: {images_dir}"
    assert json_path.exists(), f"JSON file not found: {json_path}"

    with open(json_path, 'r', encoding='utf-8') as f:
        coco = json.load(f)

    images = {img['id']: img for img in coco.get('images', [])}
    anns_by_image = defaultdict(list)
    for ann in coco.get('annotations', []):
        anns_by_image[ann['image_id']].append(ann)

    categories = coco.get('categories', [])
    categories = sorted(categories, key=lambda x: x['id'])
    catid_to_name = {c['id']: c['name'] for c in categories}
    catid_to_idx = {c['id']: i for i, c in enumerate(categories)}
    class_names = [catid_to_name[c['id']] for c in categories]


    classes_file = images_dir / 'classes.txt'
    with open(classes_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(class_names))
    print(f'✅ Wrote classes.txt: {classes_file}')

    converted, skipped, errors = 0, 0, []

    for img_id, img_info in images.items():
        fname = img_info.get('file_name')
        if not fname:
            continue
        img_path = images_dir / fname
        if not img_path.exists():
            matches = list(images_dir.rglob(fname))
            if matches:
                img_path = matches[0]
            else:
                skipped += 1
                continue

        iw, ih = img_info.get('width'), img_info.get('height')
        if not iw or not ih:
            try:
                with Image.open(img_path) as im:
                    iw, ih = im.size
            except Exception as e:
                errors.append(f'Image size error {img_path}: {e}')
                continue

        anns = anns_by_image.get(img_id, [])
        label_lines = []
        for ann in anns:
            bbox = ann.get('bbox')
            if not bbox or len(bbox) != 4:
                continue
            x_min, y_min, w, h = bbox
            if w <= 0 or h <= 0:
                continue

            x_center = x_min + w / 2.0
            y_center = y_min + h / 2.0
            x_center_n = x_center / iw
            y_center_n = y_center / ih
            w_n = w / iw
            h_n = h / ih

            cat_id = ann.get('category_id')
            if cat_id not in catid_to_idx:
                continue
            cls_idx = catid_to_idx[cat_id]
            label_lines.append(f"{cls_idx} {x_center_n:.6f} {y_center_n:.6f} {w_n:.6f} {h_n:.6f}")

        txt_path = img_path.with_suffix('.txt')
        if label_lines:
            with open(txt_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(label_lines))
        elif save_empty:
            open(txt_path, 'w').close()
        elif txt_path.exists():
            txt_path.unlink()

        converted += 1

    print(f'Processed {converted} images (skipped {skipped}). Errors: {len(errors)}')
    if errors:
        for e in errors[:10]:
            print(' -', e)
    return class_names

In [ ]:
train_classes = coco_to_yolo_and_save(TRAIN_IMAGES, TRAIN_JSON, dataset_root=DATASET_ROOT)
valid_classes = coco_to_yolo_and_save(VALID_IMAGES, VALID_JSON, dataset_root=DATASET_ROOT)

print('\nTrain classes:', train_classes)
print('Valid classes:', valid_classes)


✅ Wrote classes.txt: /content/drive/MyDrive/Sign_detection/train/classes.txt
Processed 5140 images (skipped 0). Errors: 0
✅ Wrote classes.txt: /content/drive/MyDrive/Sign_detection/valid/classes.txt
Processed 670 images (skipped 0). Errors: 0

Train classes: ['road-signs', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']
Valid classes: ['road-signs', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']


In [ ]:
yaml_path = Path(DATASET_ROOT) / 'data.yaml'
yaml_text = f"""
train: {TRAIN_IMAGES}
val: {VALID_IMAGES}
nc: {len(train_classes)}
names: {train_classes}
"""
with open(yaml_path, 'w') as f:
    f.write(yaml_text)
print(f'✅ data.yaml created at: {yaml_path}')

✅ data.yaml created at: /content/drive/MyDrive/Sign_detection/data.yaml


In [ ]:
!python train.py --img 640 --batch 16 --epochs 30 --data {yaml_path} --weights yolov5s.pt --project {DATASET_ROOT}/runs --name sign_detect --exist-ok

Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
      22/29      4.56G    0.02442   0.009945    0.01212         45        640:  36% 115/322 [01:02<01:53,  1.82it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      22/29      4.56G    0.02443    0.00995     0.0121         43        640:  36% 116/322 [01:02<01:37,  2.12it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      22/29      4.56G    0.02449    0.00995    0.01206         40        640:  36% 117/322 [01:03<01:47,  1.91it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(